In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'sample_submission.csv']


In [2]:
train=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')

In [3]:
train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
test.head()

In [4]:
import seaborn as sns
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [5]:
def punc(df,stri):
    df[stri] = df[stri].str.lower().str.replace('[^a-z]', ' ')
    df[stri] = df[stri].str.lower().str.replace('[^\w\s]',' ')
    df[stri] = df[stri].str.lower().str.replace(r'<.*?>',' ')
    df[stri] = df[stri].str.lower().str.replace(' br ',' ')
    return df

In [6]:
train=punc(train,"question1")
train=punc(train,"question2")
#test=punc(test,"question2")
#test=punc(test,"question1")

In [ ]:
train.head()

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [7]:
test=test.dropna()
train=train.dropna()

In [ ]:
test.head()

In [8]:
import nltk
from nltk.corpus import stopwords

stop = stopwords.words('english')

In [9]:
def stopi(df,stri): 
    df[stri] = df[stri].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    return df

In [10]:
train=stopi(train,"question1")
train=stopi(train,"question2")
#test=stopi(test,"question2")
#test=stopi(test,"question1")

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
def port(df,stri):
    df[stri] = df[stri].apply(lambda x: " ".join(ps.stem(word) for word in x.split()))
    return df

In [ ]:
train=port(train,"question1")
train=port(train,"question2")

In [11]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
train['question1'] = train['question1'].apply(lambda x: " ".join(wordnet_lemmatizer.lemmatize(word) for word in x.split()))
train['question2'] = train['question2'].apply(lambda x: " ".join(wordnet_lemmatizer.lemmatize(word) for word in x.split()))

In [ ]:
train.head()

In [12]:
from difflib import SequenceMatcher
ar=[]
for index, row in train.iterrows():
    #print(row['c1'], row['c2'])
    a=SequenceMatcher(None,row['question1'],row['question2']).ratio()
    ar.append(a)
    

In [13]:
train['simi']=ar

In [ ]:
train.head()

In [14]:
train['sim'] = np.where(train['simi']==1.0, 1, 0)

In [ ]:
train.head()

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words ='english', smooth_idf=False, sublinear_tf=False, norm=None, analyzer='word')

In [ ]:
import re, math
from collections import Counter

WORD = re.compile(r'\w+')

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

text1 = 'This is a foo bar sentence .'
text2 = 'This sentence is similar to a foo bar sentence .'

vector1 = text_to_vector(text1)
vector2 = text_to_vector(text2)

cosine = get_cosine(vector1, vector2)



In [ ]:
ar=[]
for index, row in train.iterrows():
    #print(row['c1'], row['c2'])
    vector1 = text_to_vector(row['question1'])
    vector2 = text_to_vector(row['question2'])
    a=get_cosine(vector1, vector2)
    ar.append(a)
    

In [ ]:
train['simi1']=ar

In [ ]:
train.head()


In [16]:
x=train.ix[:,3:5]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [17]:
y=train.is_duplicate

In [ ]:
x.head()

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=0.33, random_state=42)

In [19]:
sample = X_train["question1"] + "," + X_train["question2"]

def my_tokenizer(s):
    return s.split(",")

vect = CountVectorizer(max_features=1500)
vect = CountVectorizer(analyzer='word',tokenizer=my_tokenizer, ngram_range=(1, 3), min_df=1) 
train1 = vect.fit_transform(sample.values)

In [ ]:
train1

In [20]:
sample = X_test["question1"] + "," + X_test["question2"]

test1 = vect.transform(sample.values)

In [ ]:
def my_tokenizer(s):
    return s.split(",")

vect = TfidfVectorizer(stop_words ='english', smooth_idf=False, sublinear_tf=False, norm=None, analyzer='word',tokenizer=my_tokenizer)
sample = X_train["question1"] + "," + X_train["question2"]
train1 = vect.fit_transform(sample.values)
sample = X_test["question1"] + "," + X_test["question2"]
test1 = vect.transform(sample.values)

In [ ]:
test1

In [21]:
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(train1, y_train)
predictions = clf.predict(test1)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
accuracy
score=f1_score(y_test,predictions,average='weighted')
score,accuracy

(0.8095447191560773, 0.816632312708466)

In [22]:
from sklearn.linear_model import LogisticRegression
log=LogisticRegression(penalty='l2',C=.00001)
log.fit(train1,y_train)
y_pred = log.predict(test1)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy
score=f1_score(y_test,y_pred,average='weighted')
score,accuracy

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(0.48901015931694386, 0.6316156354233032)

In [26]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(10, input_dim=626238, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train1, y_train, epochs=1, batch_size=10,validation_data=(test1, y_test))
y_pred = model.predict(test1)
scores = model.evaluate(train1, y_train)

Train on 270872 samples, validate on 133415 samples
Epoch 1/1
270872/270872 [==============================] - 1585s 6ms/step


In [29]:
scores

[0.14439214526680125, 0.977524439587702]

In [30]:
scores = model.evaluate(test1, y_test)

133415/133415 [==============================] - 776s 6ms/step


In [31]:
scores

[0.4441041476742258, 0.8124573698614069]

In [ ]:
import xgboost as xgb
model=xgb.XGBClassifier(random_state=1,learning_rate=0.01)
model.fit(train1, y_train)
y_pred = model.predict(test1)
score=f1_score(y_test,y_pred,average='weighted')
score
accuracy = accuracy_score(y_test, y_pred)
score,accuracy

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=4,oob_score=True ,random_state =42, min_samples_split=25)
rf.fit(train1, y_train)
y_pred = rf.predict(test1) 
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy

In [ ]:
from sklearn.metrics import f1_score
score=f1_score(y_test,y_pred,average='weighted')
score

In [ ]:
sample = X_train[['question1','question2']]
sample = X_train.apply(lambda col: col.str.strip())
import scipy.sparse as sp
vect = CountVectorizer(max_features=1500,ngram_range=(1, 3))
train1 = sp.hstack(sample.apply(lambda col: vect.fit_transform(col)))

In [ ]:
sample = X_test[['question1','question2']]
sample = X_test.apply(lambda col: col.str.strip())
import scipy.sparse as sp
test1 = sp.hstack(sample.apply(lambda col: vect.transform(col)))

In [ ]:
from sklearn.linear_model import LogisticRegression
log=LogisticRegression(penalty='l2',C=.00001)
log.fit(train1,y_train)
y_pred = log.predict(test1)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(train1, y_train)
predictions = clf.predict(test1)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
accuracy